# url: https://qiita.com/nekoumei/items/648726e89d05cba6f432

In [1]:
import pandas as pd
import statsmodels.api as sm
import joblib
import os

import warnings
warnings.filterwarnings('ignore')

## セレクションバイアスのあるデータの作成

In [2]:
dumped_male_df_path = '/content/male_df.joblib'
dumped_biased_df_path = '/content/biased_df.joblib'

if os.path.exists(dumped_male_df_path):
    male_df = joblib.load(dumped_male_df_path)
    biased_df = joblib.load(dumped_biased_df_path)
else:
    # セレクションバイアスのあるデータの作成
    mail_df = pd.read_csv('http://www.minethatdata.com/Kevin_Hillstrom_MineThatData_E-MailAnalytics_DataMiningChallenge_2008.03.20.csv')
    ### 女性向けメールが配信されたデータを削除したデータを作成
    male_df = mail_df[mail_df.segment != 'Womens E-Mail'].copy() # 女性向けメールが配信されたデータを削除
    male_df['treatment'] = male_df.segment.apply(lambda x: 1 if x == 'Mens E-Mail' else 0) #介入を表すtreatment変数を追加
    ## バイアスのあるデータの作成
    sample_rules = (male_df.history > 300) | (male_df.recency < 6) | (male_df.channel=='Multichannel')
    biased_df = pd.concat([
        male_df[(sample_rules) & (male_df.treatment == 0)].sample(frac=0.5, random_state=1),
        male_df[(sample_rules) & (male_df.treatment == 1)],
        male_df[(~sample_rules) & (male_df.treatment == 0)],
        male_df[(~sample_rules) & (male_df.treatment == 1)].sample(frac=0.5, random_state=1)
    ], axis=0, ignore_index=True)

## (6) バイアスのあるデータでの回帰分析

In [3]:
## 回帰分析の実行
y = biased_df.spend
X = biased_df[['treatment', 'history']]
X = sm.add_constant(X) # statsmodelsではβ0を明示的に入れてあげる必要がある
model = sm.OLS(y, X)
results = model.fit()

In [4]:
## 分析結果のレポート
summary = results.summary()
summary

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  spend   R-squared:                       0.001
Model:                            OLS   Adj. R-squared:                  0.001
Method:                 Least Squares   F-statistic:                     20.45
Date:                Fri, 15 Dec 2023   Prob (F-statistic):           1.32e-09
Time:                        12:53:26   Log-Likelihood:            -1.3312e+05
No. Observations:               31925   AIC:                         2.663e+05
Df Residuals:                   31922   BIC:                         2.663e+05
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.3413      0.147      2.327      0.020       0.054       0.629
treatment      0.9088      0.177      5.122      0.000       0.561       1.257
history        0.0011      0.000      3.096      0.002       0.000       0.002
==============================================================================
Omnibus:                    70760.532   Durbin-Watson:                   2.002
Prob(Omnibus):                  0.000   Jarque-Bera (JB):        352134568.791
Skew:                          20.807   Prob(JB):                         0.00
Kurtosis:                     515.825   Cond. No.                         833.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [5]:
## 推定されたパラメーターの取り出し
biased_reg_coef = summary.tables[1]
biased_reg_coef

,coef,std err,t,P>|t|,[0.025,0.975]
const,0.3413,0.147,2.327,0.020,0.054,0.629
treatment,0.9088,0.177,5.122,0.000,0.561,1.257
history,0.0011,0.000,3.096,0.002,0.000,0.002


## (7) RCTデータでの回帰分析とバイアスのあるデータでの回帰分析の比較

In [6]:
## RCTデータでの単回帰
y = male_df.spend
X = male_df[['treatment']]
X = sm.add_constant(X)
results = sm.OLS(y, X).fit()
rct_reg_coef = results.summary().tables[1]
rct_reg_coef

,coef,std err,t,P>|t|,[0.025,0.975]
const,0.6528,0.103,6.356,0.000,0.451,0.854
treatment,0.7698,0.145,5.300,0.000,0.485,1.055


In [7]:
## バイアスのあるデータでの単回帰
y = biased_df.spend
X = biased_df[['treatment']]
X = sm.add_constant(X)
results = sm.OLS(y, X).fit()
nonrct_reg_coef = results.summary().tables[1]
nonrct_reg_coef

,coef,std err,t,P>|t|,[0.025,0.975]
const,0.5580,0.129,4.328,0.000,0.305,0.811
treatment,0.9837,0.176,5.596,0.000,0.639,1.328


In [8]:
## バイアスのあるデータでの重回帰
y = biased_df.spend
# R lmではカテゴリ変数は自動的にダミー変数化されているのでそれを再現
X = pd.get_dummies(biased_df[['treatment', 'recency', 'channel', 'history']], columns=['channel'], drop_first=True)
X = sm.add_constant(X)
results = sm.OLS(y, X).fit()
nonrct_mreg_coef = results.summary().tables[1]
nonrct_mreg_coef

,coef,std err,t,P>|t|,[0.025,0.975]
const,0.4761,0.386,1.233,0.218,-0.281,1.233
treatment,0.8617,0.181,4.750,0.000,0.506,1.217
recency,-0.0361,0.026,-1.372,0.170,-0.088,0.015
history,0.0010,0.000,2.655,0.008,0.000,0.002
channel_Phone,-0.0079,0.310,-0.025,0.980,-0.616,0.600
channel_Web,0.2540,0.310,0.820,0.412,-0.353,0.861


## (8) OVBの確認

In [9]:
## (a) history抜きの回帰分析とパラメーターの取り出し
y = biased_df.spend
X = pd.get_dummies(biased_df[['treatment', 'recency', 'channel']], columns=['channel'], drop_first=True)
X = sm.add_constant(X)
results = sm.OLS(y, X).fit()
short_coef = results.summary().tables[1]
short_coef_df = pd.read_html(short_coef.as_html(), header=0, index_col=0)[0] #SimpleTableは扱いなれてないのでpandas DataFrameにする

## aの結果から介入効果に関するパラメーターのみを取り出す
alpha_1 = results.params['treatment'] # summaryのデータは小数点が四捨五入されているため、正確な値をとってくる


In [10]:
## (b) historyを追加した回帰分析とパラメーターの取り出し
y = biased_df.spend
X = pd.get_dummies(biased_df[['treatment', 'recency', 'channel', 'history']], columns=['channel'], drop_first=True)
X = sm.add_constant(X)
results = sm.OLS(y, X).fit()
long_coef = results.summary().tables[1]
long_coef_df = pd.read_html(long_coef.as_html(), header=0, index_col=0)[0] #SimpleTableは扱いなれてないのでpandas DataFrameにする

## bの結果から介入とhistoryに関するパラメーターを取り出す
beta_1 = results.params['treatment']
beta_2 = results.params['history']

In [11]:
## (c) 脱落した変数と介入変数での回帰分析
y = biased_df.history
X = pd.get_dummies(biased_df[['treatment', 'recency', 'channel']], columns=['channel'], drop_first=True)
X = sm.add_constant(X)
results = sm.OLS(y, X).fit()
omitted_coef = results.summary().tables[1]
omitted_coef_df = pd.read_html(omitted_coef.as_html(), header=0, index_col=0)[0] #SimpleTableは扱いなれてないのでpandas DataFrameにする
gamma_1 = results.params['treatment']

In [12]:
## OVBの確認
print(beta_2 * gamma_1)
print(alpha_1 - beta_1)

0.028816423676830048
0.028816423676825798


## (10) 入れてはいけない変数を入れてみる

In [13]:
## visitとtreatmentの相関
y = biased_df.treatment
X = pd.get_dummies(biased_df[['visit', 'channel', 'recency', 'history']], columns=['channel'], drop_first=True)
X = sm.add_constant(X)
results = sm.OLS(y, X).fit()
results.summary().tables[1]

,coef,std err,t,P>|t|,[0.025,0.975]
const,0.7153,0.011,63.968,0.000,0.693,0.737
visit,0.1509,0.008,19.820,0.000,0.136,0.166
recency,-0.0282,0.001,-35.621,0.000,-0.030,-0.027
history,0.0001,1.17e-05,9.705,0.000,9.06e-05,0.000
channel_Phone,-0.0708,0.009,-7.453,0.000,-0.089,-0.052
channel_Web,-0.0771,0.009,-8.131,0.000,-0.096,-0.059


In [14]:
# visitを入れた回帰分析を実行
y = biased_df.spend
X = pd.get_dummies(biased_df[['treatment', 'channel', 'recency', 'history', 'visit']], columns=['channel'], drop_first=True)
X = sm.add_constant(X)
results = sm.OLS(y, X).fit()
results.summary().tables[1]

,coef,std err,t,P>|t|,[0.025,0.975]
const,-0.4057,0.382,-1.062,0.288,-1.155,0.343
treatment,0.2784,0.180,1.546,0.122,-0.075,0.631
recency,0.0090,0.026,0.346,0.729,-0.042,0.060
history,0.0005,0.000,1.316,0.188,-0.000,0.001
visit,7.2368,0.246,29.368,0.000,6.754,7.720
channel_Phone,0.0978,0.306,0.320,0.749,-0.502,0.697
channel_Web,0.1160,0.306,0.380,0.704,-0.483,0.715
